<a href="https://colab.research.google.com/github/slz520f/github-basic-kadai/blob/main/master_DjangoApp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

注）

この章はVScodeを開発環境として進めてください。Colabでは正常に動作しません。

# 開発の流れを理解しよう

Djangoアプリ制作に関する教材の要約を、指定された３つのスタイルで行います。

- **目標**: 本章では、「画像判定アプリケーション」の全体像を理解することを目指します。
  - **機能**: アプリケーションは画像アップロード、プレビュー表示、画像判定表示とクリア、再アップロードと前回判定結果のクリアの機能を有します。
  - **構成**: URL設定、View、Form、予測モデル、テンプレートが主要な構成要素です。
  - **開発流れ**: Django開発環境構築、画像アップロード機能と結果表示ロジックの実装、画像判定機能の実装、画像表示機能の実装、Bootstrapによるデザイン改善、Herokuでの本番公開。

>この章では、Djangoを使った「画像判定アプリケーション」の作り方を学びます。アプリは、画像をアップロードして、それが猫か犬かを判定します。主要なパーツにはURL設定やView、フォーム、予測モデル、テンプレートがあります。開発は、まずDjangoのセットアップから始まり、画像アップロードと表示、判定機能の実装、デザインの改善、最後にHerokuで公開する手順で進めます。



# Djangoの開発環境を構築しよう

- **目標**: Djangoを用いた画像判定アプリケーション開発のための環境構築。
  - **仮想環境構築**: Pythonの仮想環境を構築し、開発環境を隔離。
  - **ライブラリインストール**: Django, TensorFlow, Pillowの指定バージョンをインストール。
  - **プロジェクトとアプリケーションひな型作成**: Djangoプロジェクト「catordog」とアプリケーション「prediction」を作成。
  - **settings.py設定**: INSTALLED_APPSに「prediction」を追加。
  - **ブラウザ動作確認**: Djangoデフォルト画面の表示を確認。


>この章では、Djangoを使ったアプリ開発の基盤を作るために、Pythonの仮想環境を作り、Djangoや他の必要なライブラリをインストールしました。プロジェクト「catordog」とアプリ「prediction」のひな型を作って、設定ファイルを調整しました。最後に、ブラウザでDjangoのデフォルト画面がちゃんと表示されるかを確認しました。

- **Python仮想環境の構築**
  - コマンド: `python -m venv appvenv`
  - 仮想環境のアクティベート: Windowsは `.\Scripts\activate`、Macは `source appenv/bin/activate`
- **ライブラリインストール**
  - コマンド: `pip install Django==4.2.1 tensorflow==2.12.0 Pillow==9.5.0`
  - Macでのエラー対応: `pip install Django==4.2.1 tensorflow==2.13.0rc1 Pillow==9.5.0`
- **Djangoプロジェクト作成**
  - コマンド: `django-admin startproject catordog`
- **Djangoアプリケーションひな型作成**
  - コマンド: `python manage.py startapp prediction`
- **settings.py設定**
  - `INSTALLED_APPS`に「prediction」を追加
- **ブラウザでの動作確認**
  - コマンド: `python manage.py runserver`
  - ブラウザアクセス: `http://127.0.0.1:8000/`

これらのステップにより、Django開発の基本的な環境が整います。

# 画像アップロード機能と結果表示のロジックを実装しよう

- **目標**:
  - 画像アップロードのフォーム機能の実装。
  - 画像をアップロードした際、ランダムな結果を表示するロジックの実装。
- **プロセス**:
  - **フォームの作成**: `ImageUploadForm`クラスを定義し、画像ファイルアップロード用のフォームを作成。
  - **Viewの作成**: `predict`関数を定義し、GETとPOSTリクエストの処理を実装。POSTリクエストでフォームの内容を受け取り、ランダムな結果を生成。
  - **テンプレートの作成**: `home.html`を作成し、フォームと判定結果を表示。
  - **URL設定**: `urls.py`を編集し、URLと`predict`関数を結びつける。
  - **動作確認**: ブラウザでフォームの表示とランダムな判定結果の表示を確認。

この章では、画像アップロード用のフォームと、アップロードされた画像に基づいてランダムな「猫」か「犬」の判定結果を表示する機能を作りました。まずはフォームを作って、次に画像をアップロードしたときにランダムに結果が出るようにしました。それから、このフォームと結果を表示するHTMLページを作り、最後にブラウザでちゃんと動くか確認しました。

- **フォームクラスの作成 (`forms.py`)**:
  ```python
  from django import forms

  class ImageUploadForm(forms.Form):
      image = forms.ImageField()
  ```
  画像アップロードフォームクラスを定義。

- **Viewの作成 (`views.py`)**:
  ```python
  from django.shortcuts import render
  from .forms import ImageUploadForm
  import random

  def predict(request):
      if request.method == 'GET':
          form = ImageUploadForm()
          return render(request, 'home.html', {'form': form})
      elif request.method == 'POST':
          form = ImageUploadForm(request.POST, request.FILES)
          if form.is_valid():
              # 画像判定ロジックは後続章で実装
              prediction = random.choice(["猫", "犬"])
              return render(request, 'home.html', {'form': form, 'prediction': prediction})
  ```
  GETリクエストではフォームを表示、POSTリクエストではランダムな判定結果を表示。

- **テンプレートの作成 (`home.html`)**:
  ```html
  <!DOCTYPE html>
  <html lang="jp">
  <head>
      <meta charset="UTF-8">
      <title>画像判定アプリ</title>
  </head>
  <body>
      <form method="post" enctype="multipart/form-data">
          {% csrf_token %}
          {{ form.as_p }}
          <button type="submit">判定</button>
      </form>
      {% if prediction %}
          <p>判定結果: {{ prediction }}</p>
      {% endif %}
  </body>
  </html>
  ```
  フォームと判定結果を表示するHTMLテンプレート。

- **URL設定 (`urls.py`)**:


 ```python
  from django.urls import path
  from prediction.views import predict

  urlpatterns = [
      path('', predict, name='predict'),
  ]
  ```
  ルートURLを`predict`関数に関連付け。

# 画像判定機能を実装しよう

- **目標**: アップロードされた画像に対して、予測モデルを用いて「猫」か「犬」かの判定を行い、結果を表示する。
- **プロセス**:
  - **予測モデルの配置**: 学習済みの予測モデル「model.h5」をアプリケーション内に配置。
  - **Viewの改修**: 画像の前処理を実装し、予測モデルを使用して判定を行うロジックを追加。
  - **ブラウザでの動作確認**: 実際に画像をアップロードし、予測結果が適切に表示されることを確認。

>今回の章では、アップロードされた画像を「猫」か「犬」か判定する機能を作りました。最初に、以前作った予測モデルをアプリに入れて、それから画像を予測モデルで判定するためのコードをViewに書きました。最後に、実際に画像をアップロードして、ちゃんと判定結果が出るかブラウザで確認します。

- **予測モデルの配置**:
  - 予測モデル「model.h5」を「prediction/models」フォルダに配置。

- **Viewの改修 (`views.py`)**:
  ```python
  from django.shortcuts import render
  from .forms import ImageUploadForm
  from django.conf import settings
  from tensorflow.keras.models import load_model
  from tensorflow.keras.preprocessing.image import load_img, img_to_array
  from io import BytesIO
  import os

  def predict(request):
      if request.method == 'POST':
          form = ImageUploadForm(request.POST, request.FILES)
          if form.is_valid():
              img_file = form.cleaned_data['image']
              img_file = BytesIO(img_file.read())
              img = load_img(img_file, target_size=(256, 256))
              img_array = img_to_array(img)
              img_array = img_array.reshape((1, 256, 256, 3))
              img_array = img_array / 255
              model_path = os.path.join(settings.BASE_DIR, 'prediction', 'models', 'model.h5')
              model = load_model(model_path)
              result = model.predict(img_array)
              prediction = '猫' if result[0][0] > result[0][1] else '犬'
              return render(request, 'home.html', {'form': form, 'prediction': prediction})
  ```
  予測モデルを用いてアップロードされた画像の判定を行い、結果を表示。

- **ブラウザでの動作確認**:
  - Webサーバ起動: `python manage.py runserver`
  - 画像アップロードと判定結果の確認をブラウザで実施。

# 画像表示機能を実装しよう

- **目標**:
  - アップロードされた画像のプレビュー機能の実装。
  - 判定結果として、判定対象となった画像の表示。
  - 新たに画像判定を開始する際、前回の判定結果をクリアする機能の実装。
- **プロセス**:
  - **プレビュー機能の実装**: jQueryを用いて、アップロードされた画像のプレビュー表示機能を実装。
  - **判定結果での画像表示機能**: 判定結果と一緒に、判定対象となった画像を表示。
  - **判定結果のクリア機能**: 新たな画像をアップロードする際、前回の判定結果をクリアする機能をJavaScriptで実装。


この章では、画像をアップロードするとその画像がどう見えるかをプレビューできるようにしました。これで、判定する画像が正しいか確認できます。判定結果が出ると、その結果と一緒に判定した画像も表示されるようにしました。そして、新しい画像をアップロードする時は、前の判定結果が消えるようにしました。


- **プレビュー機能のJavaScriptコード (`home.html`)**:
  ```javascript
  $(function() {
      $('#id_image').on('change', function() {
          var reader = new FileReader();
          reader.onload = function () {
              $("#preview").attr('src', reader.result);
              $("#preview").show();
          }
          reader.readAsDataURL(this.files[0]);
      });
  });
  ```
  アップロードされた画像をプレビュー表示するためのコード。ファイル選択時にプレビューが表示される。

- **判定結果での画像表示機能**:
  - **Viewの修正 (`views.py`)**:
    ```python
    def predict(request):
        # ...
        if form.is_valid():
            # ...
            return render(request, 'home.html', {'form': form, 'prediction': prediction, 'img_data': img_data})
    ```
    判定結果を表示する際、アップロードした画像データも一緒に渡す。

  - **HTMLテンプレートの修正 (`home.html`)**:
    ```html
    {% if prediction %}
        <p>あなたのアップロードした画像は「{{ prediction }}」でしょう。</p>
        <img src="{{ img_data }}">
    {% endif %}
    ```
    判定結果と一緒にアップロードした画像も表示。

- **判定結果のクリア機能のJavaScriptコード (`home.html`)**:
  ```javascript
  $(function() {
      $('#id_image').on('change', function() {
          $("#prediction").text('');
          // その他のコード
      });
  });
  ```
  新しい画像をアップロードする際に、前回の判定結果をクリアするためのコード。

# Bootstrapでデザインを整えよう

- **目標**:
  - Bootstrapを使用してアプリケーションのデザインを改善。
- **プロセス**:
  - **Bootstrapリンクの追加**: HTMLファイルにBootstrapのCSSリンクを追加。
  - **ビューポート設定**: レスポンシブデザインのためのmetaタグの設定。
  - **class属性によるデザイン適用**: HTML要素にBootstrapのクラスを追加しデザインを適用。

>この章では、Bootstrapを使ってアプリの見た目をきれいにしました。まずはBootstrapのリンクをHTMLに入れて、ページがきちんと見えるようにビューポートを設定しました。それから、ボタンやフォームにBootstrapのクラスを追加して、全体的に見やすくしました。

- **BootstrapのCSSリンク追加 (`home.html`)**:
  ```html
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  ```
  Bootstrapのデザイン機能を利用するための基本リンク。

- **ビューポートのmetaタグ設定 (`home.html`)**:
  ```html
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  ```
  レスポンシブデザインをサポートするためのビューポート設定。

- **class属性によるデザイン適用 (`home.html`)**:
  - フォーム部分のデザイン:
    ```html
    <div class="container py-5 text-center">
      <h2 class="mb-4">犬か猫かを画像判定します</h2>
      <form method="post" enctype="multipart/form-data">
        <div class="mb-3">
          <!-- ... -->
        </div>
        <button type="submit" class="btn btn-primary btn-lg">判定</button>
      </form>
    </div>
    ```
    フォームとボタンにBootstrapのクラスを適用し、見た目を改善。

  - 判定結果表示部分のデザイン:
    ```html
    <div id="prediction">
      <h2 class="mb-4">判定結果</h2>
      <p>あなたのアップロードした画像は「<strong>{{ prediction }}</strong>」でしょう。</p>
      <img src="{{ img_data }}" class="img-fluid mb-3">
    </div>
    ```
    判定結果と画像表示部分にBootstrapのクラスを適用。

- **目標**:
  - アップロードされた画像のプレビュー機能の実装。
  - 判定結果として、判定対象となった画像の表示。
  - 新たに画像判定を開始する際、前回の判定結果をクリアする機能の実装。
- **プロセス**:
  - **プレビュー機能の実装**: jQueryを用いて、アップロードされた画像のプレビュー表示機能を実装。
  - **判定結果での画像表示機能**: 判定結果と一緒に、判定対象となった画像を表示。
  - **判定結果のクリア機能**: 新たな画像をアップロードする際、前回の判定結果をクリアする機能をJavaScriptで実装。


この章では、画像をアップロードするとその画像がどう見えるかをプレビューできるようにしました。これで、判定する画像が正しいか確認できます。判定結果が出ると、その結果と一緒に判定した画像も表示されるようにしました。そして、新しい画像をアップロードする時は、前の判定結果が消えるようにしました。

- **プレビュー機能のJavaScriptコード (`home.html`)**:
  ```javascript
  $(function() {
      $('#id_image').on('change', function() {
          var reader = new FileReader();
          reader.onload = function () {
              $("#preview").attr('src', reader.result);
              $("#preview").show();
          }
          reader.readAsDataURL(this.files[0]);
      });
  });
  ```
  アップロードされた画像をプレビュー表示するためのコード。ファイル選択時にプレビューが表示される。

- **判定結果での画像表示機能**:
  - **Viewの修正 (`views.py`)**:
    ```python
    def predict(request):
        # ...
        if form.is_valid():
            # ...
            return render(request, 'home.html', {'form': form, 'prediction': prediction, 'img_data': img_data})
    ```
    判定結果を表示する際、アップロードした画像データも一緒に渡す。

  - **HTMLテンプレートの修正 (`home.html`)**:
    ```html
    {% if prediction %}
        <p>あなたのアップロードした画像は「{{ prediction }}」でしょう。</p>
        <img src="{{ img_data }}">
    {% endif %}
    ```
    判定結果と一緒にアップロードした画像も表示。

- **判定結果のクリア機能のJavaScriptコード (`home.html`)**:
  ```javascript
  $(function() {
      $('#id_image').on('change', function() {
          $("#prediction").text('');
          // その他のコード
      });
  });
  ```
  新しい画像をアップロードする際に、前回の判定結果をクリアするためのコード。

# [課題]画像判定アプリケーションを作成しよう

- **目標**:
  - Bootstrapを使用してアプリケーションのデザインを改善。
- **プロセス**:
  - **Bootstrapリンクの追加**: HTMLファイルにBootstrapのCSSリンクを追加。
  - **ビューポート設定**: レスポンシブデザインのためのmetaタグの設定。
  - **class属性によるデザイン適用**: HTML要素にBootstrapのクラスを追加しデザインを適用。


この章では、Bootstrapを使ってアプリの見た目をきれいにしました。まずはBootstrapのリンクをHTMLに入れて、ページがきちんと見えるようにビューポートを設定しました。それから、ボタンやフォームにBootstrapのクラスを追加して、全体的に見やすくしました。

- **BootstrapのCSSリンク追加 (`home.html`)**:
  ```html
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  ```
  Bootstrapのデザイン機能を利用するための基本リンク。

- **ビューポートのmetaタグ設定 (`home.html`)**:
  ```html
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  ```
  レスポンシブデザインをサポートするためのビューポート設定。

- **class属性によるデザイン適用 (`home.html`)**:
  - フォーム部分のデザイン:
    ```html
    <div class="container py-5 text-center">
      <h2 class="mb-4">犬か猫かを画像判定します</h2>
      <form method="post" enctype="multipart/form-data">
        <div class="mb-3">
          <!-- ... -->
        </div>
        <button type="submit" class="btn btn-primary btn-lg">判定</button>
      </form>
    </div>
    ```
    フォームとボタンにBootstrapのクラスを適用し、見た目を改善。

  - 判定結果表示部分のデザイン:
    ```html
    <div id="prediction">
      <h2 class="mb-4">判定結果</h2>
      <p>あなたのアップロードした画像は「<strong>{{ prediction }}</strong>」でしょう。</p>
      <img src="{{ img_data }}" class="img-fluid mb-3">
    </div>
    ```
    判定結果と画像表示部分にBootstrapのクラスを適用。

この課題の模範回答は、Djangoを使ったWebアプリケーションで、学習済みのVGG16モデルを使用して画像を判定するプログラムです。以下は、このアプリケーションの基本的な実装手順を示したコードの例です。

### 基本的なステップ

1. **Djangoプロジェクトとアプリケーションのセットアップ**
2. **VGG16モデルの統合**
3. **画像アップロード機能の実装**
4. **画像判定機能の実装**
5. **結果表示機能の実装**
6. **追加機能とデザインの改善（オプション）**

### コード例

**Step 1: Djangoプロジェクトとアプリケーションのセットアップ**
```python
# Djangoプロジェクトのセットアップ
# Djangoアプリケーションの作成（例: 'photoidentify'）
```

**Step 2: VGG16モデルの統合**
```python
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

model = VGG16(weights='imagenet')

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    predictions = model.predict(img_array)
    return decode_predictions(predictions, top=5)[0]
```

**Step 3: 画像アップロード機能の実装**
```python
# Djangoのforms.pyで画像アップロードフォームを作成
# HTMLテンプレートでフォームをレンダリング
```

**Step 4: 画像判定機能の実装**
```python
# views.pyで画像アップロードと判定結果のロジックを実装
```

**Step 5: 結果表示機能の実装**
```python
# 判定結果をHTMLテンプレートに渡す
# 判定結果を表示するためのHTMLとCSSを作成
```

### 実装注意点
- VGG16モデルの入力と出力形式を理解し、適切に処理する。
- DjangoのMVCモデルに沿った実装を行う。
- ユーザインターフェースは直感的でわかりやすくする。

このコード例は概要を示しているため、実際の開発ではさらに詳細な設定やエラーハンドリング、ユーザインターフェースのデザインなどが必要になります。また、追加のJavaScript機能やBootstrapによるデザインの改善も考慮すると良いでしょう。

# Herokuでアプリを公開しよう

- **目標**:
  - Herokuに画像判定アプリをデプロイし、インターネット上で公開。
- **プロセス**:
  - **動作確認**: ローカル環境でアプリが正しく動作することを確認。
  - **デプロイの準備**: Git環境の設定、HerokuアカウントとCLIの準備。
  - **設定ファイルの作成と編集**: `Procfile`、`.gitignore`、`runtime.txt`、`requirements.txt`を作成・編集。
  - **Herokuへのデプロイ**: `git push`コマンドを使用してHerokuにアプリをデプロイ。
  - **公開アプリの確認**: デプロイされたアプリをブラウザで確認。


この章では、Herokuで作った画像判定アプリをネットに出しました。まずローカルでちゃんと動くか確認して、GitとHerokuの設定をしました。それから、Herokuに必要な`Procfile`や他のファイルを作って設定を変えて、最後に`git push`でHerokuにアップしました。最後にブラウザで新しいアプリをチェックしました。

- **Herokuにログイン**:
  ```bash
  heroku login -i
  ```
  Herokuにログインするためのコマンド。

- **Herokuアプリの作成**:
  ```bash
  heroku apps:create -a アプリ名
  ```
  新しいHerokuアプリを作成。

- **Herokuアプリの設定変更**:
  ```bash
  heroku config:set DISABLE_COLLECTSTATIC=1
  ```
  Herokuに特有の設定を適用。

- **.gitignoreファイルの作成**:
  ```plaintext
  .DS_Store
  env/
  *.log
  *.pot
  *.pyc
  __pycache__
  local_settings.py
  db.sqlite3
  media
  ```
  Herokuに不要なファイルやディレクトリをGitトラッキングから除外。

- **Procfileファイルの作成**:
  ```plaintext
  web: gunicorn --workers=1 catordog.wsgi --timeout 300 --log-file -
  ```
  Herokuでアプリを実行するための設定。

- **runtime.txtファイルの作成**:
  ```plaintext
  python-3.9.17
  ```
  Herokuで使用するPythonのバージョンを指定。

- **requirements.txtファイルの作成**:
  ```plaintext
  Django==4.2.1
  tensorflow-cpu==2.11.0
  Pillow==9.5.0
  gunicorn==20.1.0
  ```
  Herokuで必要なパッケージとそのバージョンを指定。

- **Herokuへアプリをデプロイ**:
  ```bash
  git add .
  git commit -m "deploy to heroku"
  git branch -M main
  git push heroku main
  ```
  Gitを使用してHerokuにアプリをデプロイ。